In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from pathlib import Path

import argparse
import yaml
import json

def get_all_paths(path):
    print("Collecting all paths")
    paths = []
    for fpath in path.iterdir():
        paths.append(fpath)
    return paths

def get_gpath_dict():
    paths = yaml.safe_load(open('/home2/sisodiya.bhoomendra/github/contrastive_learning/data/paths.yaml','r'))
    posix_path = {}
    for k,v in paths.items():
        posix_path[k] = Path(v)
    return posix_path


def judg2text():
    pass

def query2text():
    pass


# if __name__=='__main__':

    # parser = argparse.ArgumentParser()
    # parser.add_argument('--debug',help='Get in to debug mode',action='store_true')
    # parser.add_argument('--idx',help='index of file that will be cleaned',type=int)
    # args = parser.parse_args()
gpath = get_gpath_dict()
dpaths = get_all_paths(gpath['path_clean'])
qpaths = get_all_paths(Path('../../Judgment_retrival_Fire_2017/data/raw/Current_Cases/'))
# if args.debug:
    # print(qpaths)

In [4]:
dnames = []
dtext = []

for path in tqdm(dpaths,desc='All judgement Loading'):
    file = open(path,'r')
    judg = json.load(file)
    file.close()
    actual_paras = judg.pop('headnote')
    actual_paras.extend(judg['paragraphs'])
    out = ' '.join(actual_paras).strip()
    # print(out)
    dnames.append(path.name)
    dtext.append(out.lower())

Current Cases Loading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 2081.91it/s]


Exception: ../../Judgment_retrival_Fire_2017/data/raw/Current_Cases/current_case_0059.txt is opened in rb
(53897, 366840) (200, 366840)
27580
20624
21931
40073
52774
9914
39435
715
16254
47811
12168
16646
48781
34415
30223
7437
52883
52040
9950
21861
11504
50159
28784
35022
38250
28486
14060
6876
18342
51299
49192
22255
49540
11177
47592
7160
16263
7370
9010
39985
42972
53790
14633
31338
42738
33378
46943
16328
29748
47664
37334
4070
24534
24472
37034
12567
14709
48875
27105
17728
14171
32147
788
5749
1620
2918
43985
7924
42098
52107
29903
46842
32936
31855
32059
15747
22866
21153
34341
1279
21133
41851
11317
29915
37762
31313
5925
4233
16026
30109
48891
39177
7897
27156
12951
48190
16937
12756
3978
16299
6694
13194
28520
28298
33785
17947
28630
37549
28687
18137
46332
825
15645
51182
3840
45398
36976
30682
50063
39861
31343
16755
34695
13168
22972
51502
47433
9658
9672
36525
43749
15634
3455
50401
46131
39180
16895
43733
12982
14171
2149
52318
19035
4928
47608
39614
29748
44788
38007


IndexError: list index out of range

In [14]:
qnames = []
qtext = []

for path in tqdm(qpaths,desc='Current Cases Loading'):
    try:    
        file = open(path,'r')
        qtext.append(file.read().lower())
        file.close()
    except:
        print(f"Exception: {path} is opened in rb")
        file = open(path,'rb')
        qtext.append(str(file.read()).lower())
        file.close()
    
    qnames.append(path.name)

vct = TfidfVectorizer()
dout = vct.fit_transform(dtext)
qout = vct.transform(qtext)

print(dout.shape,qout.shape)
simi  = qout.dot(dout.T) # 200 * x
idxs = simi.argmax(axis=1)
# print(idxs)
idxs = idxs.flatten().tolist()[0]
matching = []
for i in range(len(qnames)):
    print(idxs[i])
    matching.append((qnames[i],dnames[idxs[i]]))
print(matching)

Current Cases Loading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 1531.90it/s]


Exception: ../../Judgment_retrival_Fire_2017/data/raw/Current_Cases/current_case_0059.txt is opened in rb
(53897, 366840) (200, 366840)
27580
20624
21931
40073
52774
9914
39435
715
16254
47811
12168
16646
48781
34415
30223
7437
52883
52040
9950
21861
11504
50159
28784
35022
38250
28486
14060
6876
18342
51299
49192
22255
49540
11177
47592
7160
16263
7370
9010
39985
42972
53790
14633
31338
42738
33378
46943
16328
29748
47664
37334
4070
24534
24472
37034
12567
14709
48875
27105
17728
14171
32147
788
5749
1620
2918
43985
7924
42098
52107
29903
46842
32936
31855
32059
15747
22866
21153
34341
1279
21133
41851
11317
29915
37762
31313
5925
4233
16026
30109
48891
39177
7897
27156
12951
48190
16937
12756
3978
16299
6694
13194
28520
28298
33785
17947
28630
37549
28687
18137
46332
825
15645
51182
3840
45398
36976
30682
50063
39861
31343
16755
34695
13168
22972
51502
47433
9658
9672
36525
43749
15634
3455
50401
46131
39180
16895
43733
12982
14171
2149
52318
19035
4928
47608
39614
29748
44788
38007
